In [252]:
from os import getenv
from pymongo import MongoClient
from pandas import DataFrame, Series, concat
from dotenv import load_dotenv

In [165]:
filepath = 'Users/jasongersing/PycharmProjects/fastApiProject/.env'

In [168]:
load_dotenv(filepath)

True

In [194]:
url = getenv('MONGO_URL')
database = "thruTheGrapevine"
collection_GB = "GrapeBuyers"
collection_GS = "GrapeSellers"

In [195]:
url = url[1:-1]

In [280]:
db_GB = MongoClient(url)[database][collection_GB]


In [281]:
db_GB.count_documents({})

100

In [240]:
db_GS = MongoClient(url)[database][collection_GS]

In [241]:
db_GS.count_documents({})

20

In [282]:
grape_buyers = DataFrame(db_GB.find(projection={"_id": False, "grapes_seeking": True, "volume_seeking": True}))

In [283]:
grape_buyers = grape_buyers.explode(column=["grapes_seeking", "volume_seeking"])

In [284]:
grape_buyers.rename(columns={"grapes_seeking": "grapes", "volume_seeking": "volume"}, inplace=True)

In [285]:
grape_buyers["side"] = "Buyer"

In [286]:
grape_buyers["volume"] = Series([int(num) for num in grape_buyers["volume"]])

In [287]:
grape_buyers

,grapes,volume,side
0,Mourvedre,50,Buyer
0,Pinot blanc,50,Buyer
0,Nebbiolo,50,Buyer
0,Syrah,50,Buyer
1,Syrah,10,Buyer
...,...,...,...
98,Marsanne,45,Buyer
99,Viognier,50,Buyer
99,Petit Verdot,50,Buyer
99,Marsanne,50,Buyer


In [288]:
grape_buyers = grape_buyers.groupby(["grapes", "side"]).agg({"volume": "sum"}).reset_index()

In [289]:
grape_buyers

,grapes,side,volume
0,Cabernet Sauvignon,Buyer,395
1,Chardonnay,Buyer,185
2,Granache,Buyer,545
3,Marsanne,Buyer,435
4,Merlot,Buyer,220
5,Mourvedre,Buyer,255
6,Nebbiolo,Buyer,390
7,Petit Verdot,Buyer,310
8,Pinot blanc,Buyer,355
9,Pinot gris,Buyer,555


In [290]:
grape_sellers = DataFrame(db_GS.find(projection={"_id": False, "grapes_selling": True, "volume_selling": True}))

In [291]:
grape_sellers = grape_sellers.explode(column=["grapes_selling", "volume_selling"])

In [292]:
grape_sellers.rename(columns={"grapes_selling": "grapes", "volume_selling": "volume"}, inplace=True)

In [293]:
grape_sellers["side"] = "Seller"

In [294]:
grape_sellers["volume"] = Series([int(num) for num in grape_sellers["volume"]])

In [295]:
grape_sellers = grape_sellers.groupby(["grapes", "side"]).agg({"volume": "sum"}).reset_index()

In [296]:
grape_sellers

,grapes,side,volume
0,Cabernet Sauvignon,Seller,35
1,Chardonnay,Seller,235
2,Granache,Seller,15
3,Marsanne,Seller,45
4,Merlot,Seller,170
5,Mourvedre,Seller,210
6,Nebbiolo,Seller,55
7,Petit Verdot,Seller,85
8,Pinot blanc,Seller,25
9,Pinot gris,Seller,45


In [297]:
total = concat([grape_buyers, grape_sellers])

In [298]:
total

,grapes,side,volume
0,Cabernet Sauvignon,Buyer,395
1,Chardonnay,Buyer,185
2,Granache,Buyer,545
3,Marsanne,Buyer,435
4,Merlot,Buyer,220
5,Mourvedre,Buyer,255
6,Nebbiolo,Buyer,390
7,Petit Verdot,Buyer,310
8,Pinot blanc,Buyer,355
9,Pinot gris,Buyer,555


In [299]:
import requests
from altair import Chart

In [301]:
chart = requests.get('http://127.0.0.1:8000/graph/df-grapes-by-side').json()
Chart.from_dict(chart)

alt.Chart(...)